In [ ]:
import socket
import requests
import traceback
import time

# socket de UDP
udp_send_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, proto=socket.IPPROTO_UDP)

# socket RAW de citire a răspunsurilor ICMP
icmp_recv_socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_ICMP)
icmp_recv_socket.settimeout(3)

# Funcție pentru obținerea locației unui IP
def get_ip_location(ip):
    try:
        response = requests.get(f'http://ip-api.com/json/{ip}')
        data = response.json()
        return data['city'], data['regionName'], data['country']
    except Exception as e:
        print(f"Error fetching location for IP {ip}: {e}")
        return None, None, None

def traceroute(ip, port=33434, max_hops=30):
    for ttl in range(1, max_hops + 1):
        udp_send_sock.setsockopt(socket.IPPROTO_IP, socket.IP_TTL, ttl)
        udp_send_sock.sendto(b'salut', (ip, port))

        addr = None
        try:
            data, addr = icmp_recv_socket.recvfrom(63535)
            addr = addr[0]
            city, region, country = get_ip_location(addr)
            print(f"TTL={ttl}, IP={addr}, City={city}, Region={region}, Country={country}")
        except socket.timeout:
            print(f"TTL={ttl}, Request timed out")
        except Exception as e:
            print(f"TTL={ttl}, Error: {e}")
            print(traceback.format_exc())

        if addr == ip:
            print("Destination reached!")
            break

# Lista de site-uri pentru teste
sites = ['www.baidu.cn', 'www.google.co.za', 'www.google.com.au']

# Obținere IP-uri pentru site-uri
site_ips = {site: socket.gethostbyname(site) for site in sites}

# Executare traceroute pentru fiecare site
results = {}
for site, ip in site_ips.items():
    print(f"\nTraceroute for {site} ({ip}):")
    results[site] = traceroute(ip)

# Salvare rezultate într-un fișier
with open('traceroute_results.txt', 'w') as f:
    for site, result in results.items():
        f.write(f"Traceroute for {site}:\n{result}\n")

# Plotarea rutelor pe hartă
import plotly.graph_objs as go
from plotly.offline import plot

map_data = []

for site, ip in site_ips.items():
    print(f"Plotting route for {site} ({ip})")
    latitudes = []
    longitudes = []
    for ttl in range(1, 31):
        udp_send_sock.setsockopt(socket.IPPROTO_IP, socket.IP_TTL, ttl)
        udp_send_sock.sendto(b'salut', (ip, 33434))

        try:
            data, addr = icmp_recv_socket.recvfrom(63535)
            addr = addr[0]
            response = requests.get(f'http://ip-api.com/json/{addr}')
            location = response.json()
            latitudes.append(location['lat'])
            longitudes.append(location['lon'])
            if addr == ip:
                break
        except:
            continue

    trace = go.Scattergeo(
        locationmode='ISO-3',
        lon=longitudes,
        lat=latitudes,
        mode='lines+markers',
        marker=dict(size=6, symbol='circle', color='red'),
        line=dict(width=2, color='blue'),
        name=site
    )
    map_data.append(trace)

layout = go.Layout(
    title='Traceroute Paths',
    showlegend=True,
    geo=dict(
        projection=dict(type='equirectangular'),
        showland=True,
        landcolor='rgb(217, 217, 217)',
        subunitwidth=1,
        countrywidth=1,
        subunitcolor='rgb(255, 255, 255)',
        countrycolor='rgb(255, 255, 255)'
    ),
)

fig = go.Figure(data=map_data, layout=layout)
plot(fig, filename='traceroute_map.html')



Traceroute for www.baidu.cn (183.240.99.24):
Error fetching location for IP 172.28.0.1: 'city'
TTL=1, IP=172.28.0.1, City=None, Region=None, Country=None
TTL=2, Request timed out
TTL=3, Request timed out
TTL=4, Request timed out
TTL=5, Request timed out
TTL=6, Request timed out
TTL=7, Request timed out
TTL=8, Request timed out
TTL=9, Request timed out
TTL=10, Request timed out
TTL=11, Request timed out
TTL=12, Request timed out
TTL=13, Request timed out
TTL=14, Request timed out
TTL=15, Request timed out
TTL=16, Request timed out
TTL=17, Request timed out
TTL=18, Request timed out
TTL=19, Request timed out
TTL=20, Request timed out
TTL=21, Request timed out
TTL=22, Request timed out
TTL=23, Request timed out
TTL=24, Request timed out
TTL=25, Request timed out
TTL=26, Request timed out
TTL=27, Request timed out
TTL=28, Request timed out
TTL=29, Request timed out
TTL=30, Request timed out

Traceroute for www.google.co.za (173.194.216.94):
Error fetching location for IP 172.28.0.1: 'city

'traceroute_map.html'